In [ ]:
import sys
sys.path.append("../models/yolov4/")

In [ ]:
from tool.utils import *
from tool.torch_utils import *
from tool.darknet2pytorch import Darknet
import argparse
from PIL import Image
import cv2

"""hyper parameters"""
use_cuda = False
img_size=800
conf_thres=0.8
nms_thres=0.4

namesfile = '../models/yolov4/cfg/coco_custom.names'
class_names = load_class_names(namesfile)

def detect_cv2(cfgfile, weightfile, imgfile):
    
    m = Darknet(cfgfile)

    #m.print_network()
    m.load_weights(weightfile)

    if use_cuda:
        m.cuda()

    num_classes = m.num_classes
    

    #img = cv2.imread(imgfile)
    img = imgfile
    sized = cv2.resize(img, (m.width, m.height))
    sized = cv2.cvtColor(sized, cv2.COLOR_BGR2RGB)
    

    for i in range(2):
        start = time.time()
        boxes = do_detect(m, sized, 0.4, 0.6, use_cuda, img_size)
        finish = time.time()

    
    return boxes[0]


In [ ]:
videopath = '../data/artifacts/videos/tienda.mp4'

%pylab inline 
import cv2
from IPython.display import clear_output

cmap = plt.get_cmap('tab20b')
colors = [cmap(i)[:3] for i in np.linspace(0, 1, 20)]

vid = cv2.VideoCapture(videopath)
#mot_tracker = Sort() 

skip_frames=20
while(True):
#for ii in range(10):
    ret, frame = vid.read()
    if skip_frames>0:
        skip_frames-=1
    else:
        skip_frames=20
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pilimg = Image.fromarray(frame)
        #detections = detect_image(pilimg)
        boxes = detect_cv2('../models/yolov4/cfg/yolov4-custom.cfg', '../models/yolov4/cfg/Yolov4_epoch20.pth', frame)
        print(boxes)
        img = np.copy(frame)
        colors = np.array([[1, 0, 1], [0, 0, 1], [0, 1, 1], [0, 1, 0], [1, 1, 0], [1, 0, 0]], dtype=np.float32)

        def get_color(c, x, max_val):
            ratio = float(x) / max_val * 5
            i = int(math.floor(ratio))
            j = int(math.ceil(ratio))
            ratio = ratio - i
            r = (1 - ratio) * colors[i][c] + ratio * colors[j][c]
            return int(r * 255)

        width = img.shape[1]
        height = img.shape[0]
        
        employees_line_x1=200
        employees_line_y1=750
        employees_line_x2=1500
        employees_line_y2=200
        color = (0, 255, 0) 
        thickness = 9
        cv2.line(frame, (employees_line_x1, employees_line_y1), (employees_line_x2, employees_line_y2), color, thickness) 
        cv2.putText(frame,  "Mostrador", (employees_line_x1, employees_line_y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 3)

        clients=0
        employees=0
            
        for i in range(len(boxes)):
            box = boxes[i]
            print(box)
            x1 = int((box[0] - box[2] / 2.0) * width)
            y1 = int((box[1] - box[3] / 2.0) * height)
            x2 = int((box[0] + box[2] / 2.0) * width)
            y2 = int((box[1] + box[3] / 2.0) * height)
            
            #It is 0 on the line, and +1 on one side, -1 on the other side.
            position = sign((employees_line_x2 - employees_line_x1) * (y1 - employees_line_y1) - (employees_line_y2 - employees_line_y1) * (x1 - employees_line_x1))

            

            if color:
                rgb = color
            else:
                rgb = (255, 0, 0)
                
            if len(box) >= 7 and class_names:
                cls_conf = box[5]
                cls_id = box[6]
                classes = len(class_names)
                offset = cls_id * 123457 % classes
                red = get_color(2, offset, classes)
                green = get_color(1, offset, classes)
                blue = get_color(0, offset, classes)
                if color is None:
                    rgb = (red, green, blue)
                cv2.putText(frame, class_names[cls_id], (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1.2, rgb, 1)
                if position>0 and class_names[cls_id]=="person":
                    employees+=1
                if position<0 and class_names[cls_id]=="person":
                    clients+=1
            #Only show people
            print(class_names[cls_id])
            if class_names[cls_id]=="person":
                cv2.rectangle(frame, (x1, y1), (x2, y2), rgb, 1)

        
        if employees > 0:
            color_alarm =(0, 255, 0) 
        elif employees==0 and clients==0:
            color_alarm =(255, 255, 0) 
        else:
            color_alarm =(255, 0, 0) 


        cv2.circle(frame, (2200,250), 100, color_alarm, -1)


        fig=figure(figsize=(24, 16))
        title("Video Stream")
        imshow(frame)
        show()
        clear_output(wait=True)